## Experiment description

The goal of this experiment is to train a TM classifier trained on 40% of the data available in Wind Farm C, and then evaluate the model on the remaining 60% of the data. The model will be evaluated on the accuracy score.

In [1]:
# Import all required libraries
import numpy as np
import pickle

from tmu.models.classification.vanilla_classifier import TMClassifier

2025-05-09 10:15:24,583 - tmu.util.cuda_profiler - WARNING - Could not import pycuda: No module named 'pycuda'
2025-05-09 10:15:24,585 - tmu.clause_bank.clause_bank_cuda - ERROR - No module named 'pycuda'
Traceback (most recent call last):
  File "/Users/kjellhaaland/Documents/GitHub/uia-master-thesis/.venv/lib/python3.12/site-packages/tmu/clause_bank/clause_bank_cuda.py", line 41, in <module>
    from pycuda._driver import Device, Context
ModuleNotFoundError: No module named 'pycuda'
2025-05-09 10:15:24,586 - tmu.clause_bank.clause_bank_cuda - WARNING - Could not import pycuda. This indicates that it is not installed! A possible fix is to run 'pip install pycuda'. Fallback to CPU ClauseBanks.


In [2]:
train_datasets = np.array([55, 81, 47, 12, 4, 18, 28, 39, 66, 15, 78, 79, 30])
test_datasets = np.array([33])

In [3]:
## Helper functions for saving the model and accuracy

# Helper function to save the model
def save_model(model, filename):
    with open(filename, "wb") as f:
        pickle.dump(model, f)


# Helper function to save the accuracy
def save_accuracy(epoch, accuracy, tp, tn, fp, fn):
    with open("accuracy.txt", "a") as f:
        f.write(f"{epoch},{accuracy},{tp},{tn},{fp},{fn}\n")


# Helper function to load dataset
def load_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_train/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_test/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def get_train_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_dataset("C", dataset) for dataset in train_datasets])
    labels = np.concatenate([load_dataset_labels("C", dataset) for dataset in train_datasets])
    return dataset, labels


def get_test_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_test_dataset("C", dataset) for dataset in test_datasets])
    labels = np.concatenate([load_test_dataset_labels("C", dataset) for dataset in test_datasets])
    return dataset, labels

In [4]:
epochs = 100
platform = "CPU"

tm = TMClassifier(
    number_of_clauses=2140,
    T=4622,
    s=23.7,
    max_included_literals=8658,
    weighted_clauses=True,
    type_iii_feedback=False,
    platform=platform,
    batch_size=250,
)

best_accuracy = 0

print(f"Running {TMClassifier} for {epochs} epochs")

(X_train, y_train) = get_train_dataset()
(X_test, y_test) = get_test_dataset()

for epoch in range(epochs):

    tm.fit(X_train, y_train)

    pred = tm.predict(X_test)

    pred_0s = np.count_nonzero(pred == 0)
    pred_1s = np.count_nonzero(pred == 1)

    print(f"Predicted 0s: {pred_0s}, Predicted 1s: {pred_1s}")

    accuracy = np.sum(pred == y_test) / len(y_test)

    # the number of false positives based on g and p
    fp = np.sum((pred == 1) & (y_test == 0))

    # the number of true negatives based on g and p
    tn = np.sum((pred == 0) & (y_test == 0))

    # the number of true positives based on g and p
    tp = np.sum((pred == 1) & (y_test == 1))

    # the number of false negatives based on g and p
    fn = np.sum((pred == 0) & (y_test == 1))

    print(f"Accuracy = {accuracy:.5f}, TP = {tp}, TN = {tn}, FP = {fp}, FN = {fn}")

    save_accuracy(epoch, accuracy, tp, tn, fp, fn)

    # Print every 20 epochs
    #if (epoch + 1) % 20 == 0:
    print(f"Epoch: {epoch + 1}, Accuracy: {accuracy:.5f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy

        print("Saving model")
        save_model(tm, "best_1.pkl")


Running <class 'tmu.models.classification.vanilla_classifier.TMClassifier'> for 100 epochs
Predicted 0s: 3302, Predicted 1s: 11
Accuracy = 0.13311, TP = 10, TN = 431, FP = 1, FN = 2871
Epoch: 1, Accuracy: 0.13311
Saving model
Predicted 0s: 3300, Predicted 1s: 13
Accuracy = 0.13372, TP = 12, TN = 431, FP = 1, FN = 2869
Epoch: 2, Accuracy: 0.13372
Saving model
Predicted 0s: 3303, Predicted 1s: 10
Accuracy = 0.13341, TP = 10, TN = 432, FP = 0, FN = 2871
Epoch: 3, Accuracy: 0.13341
Predicted 0s: 3266, Predicted 1s: 47
Accuracy = 0.12285, TP = 11, TN = 396, FP = 36, FN = 2870
Epoch: 4, Accuracy: 0.12285
Predicted 0s: 3313, Predicted 1s: 0
Accuracy = 0.13040, TP = 0, TN = 432, FP = 0, FN = 2881
Epoch: 5, Accuracy: 0.13040
Predicted 0s: 3312, Predicted 1s: 1
Accuracy = 0.13070, TP = 1, TN = 432, FP = 0, FN = 2880
Epoch: 6, Accuracy: 0.13070
Predicted 0s: 3033, Predicted 1s: 280
Accuracy = 0.19137, TP = 241, TN = 393, FP = 39, FN = 2640
Epoch: 7, Accuracy: 0.19137
Saving model
Predicted 0s: 31